<font size=4 fonc="仿宋" color=blue>第1步：导入模块与设置环境</font>

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential, layers, utils
from sklearn.model_selection import train_test_split

<font size=4 fonc="仿宋" color=blue>第2步：加载数据</font>

In [ ]:
%%time
data = pd.read_csv("../data/bike_data.csv", encoding="gbk")
data.head()

<font size=4 fonc="仿宋" color=blue>第3步：数据清洗</font>

In [ ]:
len(data.Model.unique().tolist())

In [ ]:
data['Model'] = data['Model'].astype(str).apply(lambda x: x.replace("Women's Mountain Shorts", '未知名称'))
data['Model'] = data['Model'].astype(str).apply(lambda x: x.replace("(", ""))
data['Model'] = data['Model'].astype(str).apply(lambda x: x.replace(")", ""))
data['Model'] = data['Model'].astype(str).apply(lambda x: x + " ")
data.head()

<font size=4 fonc="仿宋" color=blue>第4步：合并数据集，形成长文本</font>

In [ ]:
%%time
df = data.groupby(['OrderNumber'])['Model'].sum().reset_index()
df['Model'] = df['Model'].astype(str).apply(lambda x: x.strip())
df.head()

In [ ]:
df['count'] = df['Model'].astype(str).apply(lambda x: x.split(' ')).apply(len)
df.head()

In [ ]:
df[df['count']==1].head()

In [ ]:
texts = df[df['count']>=2].Model.tolist()
texts[:5]

In [ ]:
all_texts = texts[0]
for i in texts[1:]:
    all_texts += " " + i

In [ ]:
all_texts = list(set(all_texts.split(' ')))
all_texts.append("<PAD>")
all_texts = set(all_texts)
len(all_texts)

In [ ]:
text_to_id = {c: i for i, c in enumerate(all_texts)}
id_to_text = {i: c for i, c in enumerate(all_texts)}

In [ ]:
df['Model'] = df['Model'].astype(str).apply(lambda x: "<PAD> <PAD> <PAD> " + x)
df.head()

In [ ]:
df['Model'] = df['Model'].astype(str).apply(lambda x: x.split(' '))
df.head()

In [ ]:
x = []
y = []

for values in df['Model'].tolist():
    if len(values) == 5:
        x.append(" ".join(values[:4]))
        y.append(values[-1])
    else:
        for i in np.arange(0, len(values)-4, step=1):
            x.append(" ".join(values[i: i+4]))
            y.append("".join(values[i+4]))

In [ ]:
texts = []
for text in x:
    texts.append([text_to_id[i] for i in text.split(' ')])

label = [text_to_id[i] for i in y]
texts[:5]

In [ ]:
len(texts)

In [ ]:
texts = np.array(texts)
label = np.array(label)

In [ ]:
texts = texts / len(all_texts)

In [ ]:
x = texts.reshape(texts.shape[0], 4, 1)
x.shape

In [ ]:
y = tf.keras.utils.to_categorical(label)
y.shape

<font size=4 fonc="仿宋" color=blue>第5步：划分训练集与测试集</font>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2, 
                                                    random_state=2021)

<font size=4 fonc="仿宋" color=blue>第6步：构建模型</font>

In [ ]:
model = Sequential([
    layers.LSTM(units=800, input_shape=(4, 1), return_sequences=True),
    layers.Dropout(0.3),
    layers.LSTM(units=800, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(units=800),
    layers.Dropout(0.2),
    layers.Dense(y.shape[1], activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x, y, 
#           validation_data=(x_test, y_test),
          epochs=500, 
          batch_size=64)